In [2]:
import pickle
from tensorflow.keras.utils import to_categorical

In [3]:
x_train = pickle.load(open('x_train.pickle','rb'))
y_train = pickle.load(open('y_train.pickle','rb'))
x_test = pickle.load(open('x_test.pickle','rb'))
y_test = pickle.load(open('y_test.pickle','rb'))
x_val = pickle.load(open('x_val.pickle','rb'))
y_val = pickle.load(open('y_val.pickle','rb'))

In [13]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le.fit(y_train)
y_train_1 = le.transform(y_train)

le = LabelEncoder()
le.fit(y_test)
y_test_1 = le.transform(y_test)

le = LabelEncoder()
le.fit(y_val)
y_val_1 = le.transform(y_val)

# LOADING THE MODEL

In [16]:
from tensorflow.keras.applications import*

In [31]:
base_model = VGG19(include_top=False,weights='imagenet',input_shape=(50,50,3))

In [32]:
for layers in base_model.layers:
    layers.trainable = False
    

In [33]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 50, 50, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 50, 50, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 25, 25, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 25, 25, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 25, 25, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [34]:
feature_extractor=base_model.predict(x_train)

In [35]:
features = feature_extractor.reshape(feature_extractor.shape[0],-1)

In [36]:
X_for_SVM = features 

In [38]:
Feature_extractor = base_model.predict(x_val)

In [39]:
Features = Feature_extractor.reshape(Feature_extractor.shape[0],-1)

In [40]:
X_test_for_SVM = Features

# SVM

In [41]:
from sklearn.svm import SVC

In [42]:
Classifier = SVC(kernel='rbf', random_state=2)

In [43]:
Classifier.fit(X_for_SVM,y_train)

SVC(random_state=2)

In [46]:
prediction_SVM = Classifier.predict(X_test_for_SVM)

NameError: name 'np' is not defined

In [47]:
import numpy as np
prediction_SVM = np.array(prediction_SVM).astype(int)

In [48]:
prediction_SVM = le.inverse_transform(prediction_SVM)

In [54]:
prediction_SVM

array(['68', '0', '99', ..., '68', '68', '68'], dtype='<U3')

In [49]:
le = LabelEncoder()
le.fit(prediction_SVM)
prediction_SVM_encoded = le.transform(prediction_SVM)

In [74]:
prediction_SVM_encoded

array([62,  0, 93, ..., 62, 62, 62])

In [77]:
y_val

array(['0', '0', '0', ..., '99', '99', '99'], dtype='<U3')

In [79]:
from sklearn import metrics
print ("Accuracy = ", (metrics.accuracy_score(y_val_1, prediction_SVM_encoded))*100,'%')

Accuracy =  1.7048481619605753 %
